In [1]:
import pandas as pd
import ebcdic
import codecs

file = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

with open(file, 'rb') as ebcdic:
    data = ebcdic.read()

ascii_txt = codecs.decode(data, 'cp1140')

In [2]:
split_records = []
n  = 247 ##Unknown if this holds true for all versions of this file or for other files on TXRRC

for index in range(0, len(ascii_txt), n):
    split_records.append(ascii_txt[index : index + n])

#### Checking different fields and the formats
The lat and long data is a COBOL comp3 format decimal PIC S9(3)V9(7) which should translate to DDD.DDDDDDD

Getting values in the last position that are letters or "{" that do not make sense. From;
http://3480-3590-data-conversion.com/article-packed-fields.html
the last position should indicate the sign of the number, but there isn't a consistent decoding to make that work out.

The TXRRC map shows a 7decimal place percision when exported, but only 6 when reviewed though the info tool, these might be coming from different file sources. The 6decimal percision can be grabbed from the current field without the last letter code

In [44]:
API = None
ct = 0

sample_records = split_records[4000000:4200000]

for record in sample_records:
    if record.startswith('01'):
        API = '42'+record[2:10]
    if record.startswith('13'):
        #print(record)
        #print(ct,'API:',API,'lat:',record[132:142],'long:',record[142:152])
        try:
            #Checking to see if value for WB-PLANE-COORDINATE-EAST can be converted to number
            val = int(record[159:168])
        except:
            val = 0
            
        if val > 0:
            lat = float(str(int(record[132:135]))+'.'+str(int(record[135:141])))
            long = float('-'+str(int(record[142:145]))+'.'+str(int(record[145:151])))
            print(lat, long)
            print(ct,'API:',API,'lat:',record[132:142],'long:',record[142:152])
            print(ct,'API:',API,'Zone:',record[157:159],'Coord-E:',record[159:169],'Coord-N:',record[169:179])
        ct+=1

31.645677 -102.36856
1063 API: 4210334286 lat: 031645677F long: 102368560A
1063 API: 4210334286 Zone: 03 Coord-E: 013668190{ Coord-N: 007254970{
31.639793 -102.363698
1089 API: 4210334312 lat: 031639793F long: 102363698D
1089 API: 4210334312 Zone: 03 Coord-E: 013692980{ Coord-N: 007234250{
31.61775 -102.325234
1100 API: 4210334323 lat: 031617750A long: 102325234I
1100 API: 4210334323 Zone: 03 Coord-E: 013799500{ Coord-N: 007150400{
31.622356 -102.325796
1101 API: 4210334324 lat: 031622356F long: 102325796B
1101 API: 4210334324 Zone: 03 Coord-E: 013799750{ Coord-N: 007157700{
31.624079 -102.321936
1102 API: 4210334325 lat: 031624079G long: 102321936C
1102 API: 4210334325 Zone: 03 Coord-E: 013811900{ Coord-N: 007173630{
31.615237 -102.324366
1103 API: 4210334326 lat: 031615237D long: 102324366I
1103 API: 4210334326 Zone: 03 Coord-E: 013802000{ Coord-N: 007141200{
31.582741 -102.346673
1182 API: 4210334405 lat: 031582741E long: 102346673A
1182 API: 4210334405 Zone: 03 Coord-E: 013731690{ 

In [18]:
len(split_records)

25198781